# Pandemic Network Model:
## Predicting the spread of plant pests or pathogens using trade, environment, and pest ecology open data

This notebook provides the workflow for running the Pandemic Network Model. To run this notebook, the following are assumed:
- Cloned the Pandemic GitHub repository (git clone https://github.com/ncsu-landscape-dynamics/Pandemic_Model.git)
- Notebook launched from the notebook folder of the cloned repo
- Already have the required data downloaded and formatted (see data_aqucisition_format notebook)

## Imports

In [8]:
import os
import sys
import json
import math
import numpy as np
import pandas as pd
import geopandas
from dotenv import load_dotenv

In [2]:
os.chdir('../')
print(os.getcwd())

c:\Users\kmontgomery\github\PoPS-Global


In [43]:
from pandemic.helpers import create_trades_list
from pandemic.model_equations import (
    pandemic_multiple_time_steps,
)
from pandemic.output_files import (
    create_model_dirs,
    save_model_output,
    aggregate_monthly_output_to_annual,
    write_model_metadata,
    write_annual_output
)

## Set Paths and Environment Variables

In [10]:
# Path to folder containing .env file if not saved in current working directory
# e.g., 'H:/Shared drives/APHIS  Projects/Pandemic/Data'
env_file_path = str(input())
env_file = os.path.join(env_file_path, ".env")
load_dotenv(env_file)

True

In [4]:
load_dotenv(os.path.join('.env'))

True

In [12]:
# Read environmental variables
data_dir = os.getenv('DATA_PATH')
input_dir = os.getenv('INPUT_PATH')
out_dir = os.getenv('OUTPUT_PATH')
countries_path = os.getenv('COUNTRIES_PATH')

print(input_dir)

C:/Users/kmontgomery/OneDrive - USDA/Tuta absoluta/PoPSGlobal_Tutaabsoluta/Data/inputs/


## Set Model Parameters
Model arguments can be set with a configuration file, created in the
create_model_config notebook, or by entering in the values below

### With Configuration File

In [ ]:
# Path to model configuration file
# e.g. input_dir + '/config_files/slf_gamma4-1_6801-6804_v2/config.json'
# path_to_config_json = str(input())

In [ ]:
# Read model arguments from configuration file
# with open(path_to_config_json) as json_file:
#     config = json.load(json_file)

# Columns to drop from model output dataframe 
# cols_to_drop = config["columns_to_drop"]

# commodity_path = config["commodity_path"]
# commodity_forecast_path = config["commodity_forecast_path"]
# native_countries_list = config["native_countries_list"]
# season_dict = config["season_dict"]
# alpha = config["alpha"]
# beta = config["beta"]
# mu = config["mu"]
# lamda_c_list = config["lamda_c_list"]
# phi = config["phi"]
# w_phi = config["w_phi"]
# start_year = config["start_year"]
# stop_year = config["stop_year"]
# random_seed = config["random_seed"]
# cols_to_drop = config["columns_to_drop"]
# time_infect_units = config["transmission_lag_unit"]
# transmission_lag_type = config["transmission_lag_type"]
# time_infect = config["time_to_infectivity"]
# gamma_shape = config["transmission_lag_shape"]
# gamma_scale = config["transmission_lag_scale"]
# save_entry = config['save_entry']
# save_estab = config['save_estab']
# save_intro = config["save_intro"]
# save_country_intros = config["save_country_intros"]

### In Current Session

In [13]:
# commodity_path = f"{input_dir}/monthly_agg/6801-6804"
commodity_path = f"{input_dir}/comtrade/annual_adjusted/0702"

# Forecast path can be set to None to exclude forecast from model run
commodity_forecast_path = f"{input_dir}/comtrade/trade_forecast/annual_adjusted/0702"

# List of countries where pest is present at time T0
native_countries_list = [
    "Brazil",
    "Argentina",
    "Venezuela",
    "Bolivia",
    "Colombia",
    "Chile",
    "Peru",
    "Ecuador",
    "Paraguay",
    "Uruguay"
]

# List of months when pest can be transported
season_dict = {
        "NH_season": [
            "09",
            "10",
            "11",
            "12",
            "01",
            "02",
            "03",
            "04",
            "05",
            "06",
            "07",
            "08"
        ],
        "SH_season": [
            "09",
            "10",
            "11",
            "12",
            "01",
            "02",
            "03",
            "04",
            "05",
            "06",
            "07",
            "08"
        ]
    }

# scenario_list = []

# for i in range(2010, 2030):
#     start_scenario = [2010, 'CHN', 'USA', 'decrease', 1]
#     new_scenario = start_scenario
#     new_scenario[0] = i
#     scenario_list .append(new_scenario)
    
# for i in range(2014, 2030):
#     start_scenario = [2014, 'JPN', 'USA', 'decrease', 0.8]
#     new_scenario = start_scenario
#     new_scenario[0] = i
#     scenario_list .append(new_scenario)
    
# for i in range(2014, 2030):
#     start_scenario = [2014, 'KOR', 'USA', 'decrease', 0.8]
#     new_scenario = start_scenario
#     new_scenario[0] = i
#     scenario_list .append(new_scenario)
    
# for i in range(2020, 2030):
#     start_scenario = [2020, 'ITA', 'USA', 'decrease', 0.8]
#     new_scenario = start_scenario
#     new_scenario[0] = i
#     scenario_list .append(new_scenario)
    
# for i in range(2020, 2030):
#     start_scenario = [2020, 'TUR', 'USA', 'decrease', 0.8]
#     new_scenario = start_scenario
#     new_scenario[0] = i
#     scenario_list .append(new_scenario)

In [39]:
# Save n x n matrices for each time step where n is the number of countries, 
# and values represent the origin-destination probability of entry or 
# probability of establishment 
save_entry = False
save_estab = False
save_intro = False
save_country_intros = False

## Load Model Input Data

In [50]:
# Read formatted countries geopackage, distance matrix, and climate similarities matrix
countries = geopandas.read_file(countries_path, driver="GPKG")
distances = np.load(input_dir + '/distance_matrix.npy')
climate_similarities = np.load(input_dir + '/climate_similarities_hiiMask.npy')

In [51]:
# Read & format trade data
trades_list, file_list_filtered, code_list, commodities_available = create_trades_list(
    commodity_path=commodity_path,
    commodity_forecast_path=commodity_forecast_path,
    start_year=start_year,
    stop_year=stop_year,
    distances=distances,
)

Loading and formatting trade data...
	 ['C:/Users/kmontgomery/OneDrive - USDA/Tuta absoluta/PoPSGlobal_Tutaabsoluta/Data/inputs//comtrade/annual_adjusted\\0702']


In [52]:
# Create list of unique dates from trade data
date_list = []
for f in file_list_filtered:
    fn = os.path.split(f)[1]
    ts = str.split(os.path.splitext(fn)[0], "_")[-1]
    date_list.append(ts)
date_list.sort()
end_sim_year = date_list[-1][:4]

In [54]:
# Example trade array for formatting outputs
traded = pd.read_csv(
    file_list_filtered[0], sep=",", header=0, index_col=0, encoding="latin1"
)

In [55]:
# Checking trade array shapes
print("Length of trades list: ", len(trades_list))
for i in range(len(trades_list)):
    print("\tcommodity array shape: ", trades_list[i].shape)

Length of trades list:  1
	commodity array shape:  (31, 233, 233)


## Run Model for Selected Time Steps and Commodities

In [69]:
alpha = 0.6
beta = 0.2
mu = 0
lamda_c_list = [200]
phi = 1
w_phi = 1
start_year = 2000
stop_year = None
random_seed = None

# Transmission lag type to include (e.g., None, static, stochastic)
transmission_lag_type = "stochastic" 

# If transmission lag type is static, set time_infect as number of 
# years to delay country from becoming an origin. If lag type is
# none or stochastic, set to None
time_infect = None 
time_infect_units = "year"

# If transmission lag type is stochastic, set values to generate
# a gamma distribution. If lag type is none or static, ste to None. 
gamma_shape = 4
gamma_scale = 1

In [70]:
# Name model results: simultation name, additional description (i.e., parameter value
# or scenario tested), and run number to identify stochastic run. 
sim_name = f'tuta_test'
add_descript = 'climate_origin_match'

# First run should be run_num = 0
run_num = 4

In [71]:
print("Number of commodities: ", len([c for c in lamda_c_list if c > 0]))
print("Number of time steps: ", trades_list[0].shape[0])
for i in range(len(trades_list)):
    if len(trades_list) > 1:
        code = code_list[i]
        print("\nRunning model for commodity: ", code)
    else:
        code = code_list[0]
        print(
            "\nRunning model for commodity: ",
            os.path.basename(commodities_available[0]),
        )
    trades = trades_list[i]
    distances = distances
    locations = countries
    prob = np.zeros(len(countries.index))
    pres_ts0 = [False] * len(prob)
    infect_ts0 = np.empty(locations.shape[0], dtype="object")
    for country in native_countries_list:
        country_index = countries.index[countries["NAME"] == country][0]
        pres_ts0[country_index] = True
        # if time steps are monthly and time to infectivity is in years
        if len(date_list[0]) > 4:
            infect_ts0[country_index] = str(start_year) + "01"
        # else if time steps are annual and time to infectivity is in years
        else:
            infect_ts0[country_index] = str(start_year)
    locations["Presence"] = pres_ts0
    locations["Infective"] = infect_ts0

    sigma_h = (1 - countries["Host Percent Area"]).std()

    if len(climate_similarities.shape) == 1:
        sigma_kappa = np.std(1 - climate_similarities)
    else:
        iu1 = np.triu_indices(climate_similarities.shape[0], 1)
        sigma_kappa = np.std(1 - climate_similarities[iu1])
    
    np.random.seed(random_seed)
    lamda_c = lamda_c_list[i]

    if lamda_c > 0:
        e = pandemic_multiple_time_steps(
            trades=trades,
            distances=distances,
            locations=locations,
            climate_similarities=climate_similarities,
            alpha=alpha,
            beta=beta,
            mu=mu,
            lamda_c=lamda_c,
            phi=phi,
            sigma_h=sigma_h,
            sigma_kappa=sigma_kappa,
            w_phi=w_phi,
            start_year=start_year,
            date_list=date_list,
            season_dict=season_dict,
            transmission_lag_type=transmission_lag_type,
            #time_infect_units=time_infect_units,
            time_infect=time_infect,
            gamma_shape=gamma_shape,
            gamma_scale=gamma_scale,
            # scenario_list=scenario_list
        )


        run_prefix = f"{sim_name}_{add_descript}_{code}"

        arr_dict = {
            "prob_entry": "probability_of_entry",
            "prob_intro": "probability_of_introduction",
            "prob_est": "probability_of_establishment",
            "country_introduction": "country_introduction",
        }

        outpath = out_dir + f"/{sim_name}/{run_prefix}/run_{run_num}/"
        create_model_dirs(
            outpath=outpath,
            output_dict=arr_dict,
            write_entry_probs=save_entry, 
            write_estab_probs=save_estab,
            write_intro_probs=save_intro,
            write_country_intros=save_country_intros,
        )
        print("saving model outputs: ", outpath)
        full_out_df = save_model_output(
            model_output_object=e,
            example_trade_matrix=traded,
            outpath=outpath,
            date_list=date_list,
            write_entry_probs=save_entry,
            write_estab_probs=save_estab,
            write_intro_probs=save_intro,
            write_country_intros=save_country_intros,
            columns_to_drop=None
        )

        # If time steps are monthly, aggregate predictions to
        # annual for dashboard display
        if len(date_list[i]) > 4:
            print("aggregating monthly predictions to annual time steps...")
            aggregate_monthly_output_to_annual(
                formatted_geojson=full_out_df, outpath=outpath
            )
        
        # If time steps are annual, export the predictions
        if len(date_list[i]) == 4:
            print("exporting annual predictions...")
            write_annual_output(formatted_geojson=full_out_df, outpath=outpath)

        # Save model metadata to text file
        print("writing model metadata...")
        write_model_metadata(
            main_model_output=e[0],
            alpha=alpha,
            beta=beta,
            mu=mu,
            lamda_c_list=lamda_c_list,
            phi=phi,
            sigma_h=sigma_h,
            sigma_kappa=sigma_kappa,
            w_phi=w_phi,
            start_year=start_year,
            end_sim_year=end_sim_year,
            transmission_lag_type=transmission_lag_type,
            time_infect_units=time_infect_units,
            gamma_shape=gamma_shape,
            gamma_scale=gamma_scale,
            random_seed=random_seed,
            time_infect=time_infect,
            native_countries_list=native_countries_list,
            countries_path=countries_path,
            commodities_available=commodities_available[i], 
            commodity_forecast_path=commodity_forecast_path,
            phyto_weights=list(locations['Phytosanitary Capacity'].unique()),
            outpath=outpath, 
            run_num=run_num,
        )

    else:
        print("\tskipping as pest is not transported with this commodity")

Number of commodities:  1
Number of time steps:  31

Running model for commodity:  0702
TIME STEP:  2000
TIME STEP:  2001
TIME STEP:  2002
		 Argentina --> Paraguay
			first intro...
			time to infectious:  3
				Paraguay infective:  2005
TIME STEP:  2003
TIME STEP:  2004
TIME STEP:  2005
		 Brazil --> Paraguay
			reintroduction....
			latest time to infectious:  4
				orig: 2005 	 new: 2009
				keeping:  2005
TIME STEP:  2006
		 Chile --> Spain
			first intro...
			time to infectious:  4
				Spain infective:  2010
TIME STEP:  2007
		 Brazil --> Paraguay
			reintroduction....
			latest time to infectious:  4
				orig: 2005 	 new: 2011
				keeping:  2005
TIME STEP:  2008
TIME STEP:  2009
TIME STEP:  2010
		 Spain --> Netherlands
			first intro...
			time to infectious:  5
				Netherlands infective:  2015
		 Spain --> Portugal
			first intro...
			time to infectious:  3
				Portugal infective:  2013
TIME STEP:  2011
		 Spain --> Italy
			first intro...
			time to infectious:  7
				Italy